# 5.5 - Automatización

![automatizacion](images/automatizacion.png)

Selenium automatiza los navegadores. Principalmente, es para la automatización de aplicaciones web con fines de pruebas, pero ciertamente no se limita a eso. Las tareas aburridas de administración basadas en web pueden ser automatizadas.

Veremos como realizar estas automatizaciones y además ocultar nuestro driver para que parezca una persona.

**Librerias básicas**

In [1]:
from selenium import webdriver  # driver selenium

import time  # manejo temporal

### Opciones del driver

https://peter.sh/experiments/chromium-command-line-switches/


In [2]:
from selenium.webdriver.chrome.options import Options

In [3]:
# opciones del driver

opciones=Options()

opciones.add_argument('--start-maximized')  # empieza maximizado

#opciones.add_argument('--headlees')
opciones.headless=False               # Si es True no aparece la pantalla

opciones.add_experimental_option('excludeSwitches', ['enable-automation']) # para quitar flag de robot
opciones.add_experimental_option('useAutomationExtension', False)

opciones.add_extension('driver/adblock.crx')  # extension de adblocker

#opciones.add_argument('--incognito')

opciones.add_argument('user-data-dir=selenium')   # para guardar cookies

#opciones.add_argument('--no-proxy-server')  # sin proxy
#opciones.add_argument('proxy-server=106.122.8.54:3128')  # proxy custom

### Creando usuario random

https://pypi.org/project/fake-useragent/

In [ ]:
!pip install fake_useragent

In [ ]:
from fake_useragent import UserAgent

usuario=UserAgent().random

print(usuario)

opciones.add_argument(f'user-agent={usuario}')

### Editando el archivo chromedriver


Distil web monitor comprueba el patrón de regex  "[a-z]dc_", y el chromedriver tiene una variable que cumple con dicho patrón "cdc_". Esto hace posible la detección del driver en la red por parte de los monitores.


**Cambiando la string cdc_**

Podemos editar el chromedriver usando Vim, escribiendo en terminal:

`vim /ruta/al/chromedriver`

Una ver abierto el Vim, verás muchas **@** en el archivo. En el editor realizamos los siguientes pasos:

+ Vamos a reemplazar la string `cdc_` todas las veces que aparece escribiendo: `%s/cdc_/arm_/g`. arm_ es solo un ejemplo, se puede sustituir con cualquier string con la condición de que se mantenga la longitud, si no el driver fallará.

+ Para guardar los cambios y salir de Vim se escribe: `:wq!` y presiona Enter. Para salir sin guardar `:q!`.


In [4]:
from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()   #'driver/chromedriver'

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


### Ejemplo entrepeliculasyseries

https://entrepeliculasyseries.io/

In [5]:
from selenium.webdriver.common.by import By   # para elegir por elemento
from selenium.webdriver.support.ui import WebDriverWait # para el time sleep
from selenium.webdriver.support import expected_conditions as EC # condiciones esperadas
from selenium.webdriver import ActionChains as AC # acciones en cadena

In [6]:
url='https://entrepeliculasyseries.io/'

In [14]:
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(url)

time.sleep(5)

#busqueda
BUSQUEDA='final space'

buscar=driver.find_element_by_xpath('//*[@id="tr_live_search"]') # objeto en la web de busqueda

buscar.send_keys(BUSQUEDA)  # incrusta la string
time.sleep(3)
buscar.submit() # dale al enter

primer_res=driver.find_element_by_xpath('//*[@id="Tp-Wp"]/div/main/div/section/div[2]/ul/li/div/a')
primer_res.click()

# temporadas
TEMPORADA=2

temporadas=driver.find_element_by_xpath('//*[@id="select-season"]')
temporadas.click()

chooser=WebDriverWait(temporadas, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),'+'"'+f'Temporada {TEMPORADA}'+'"'+')]')))
chooser.click()


capitulos=driver.find_element_by_xpath(f'//*[@id="season-{TEMPORADA}"]')

capitulos=capitulos.find_elements_by_tag_name('li')
time.sleep(2)


# desde aqui al capitulo
CAPITULO=1
capitulos[CAPITULO-1].click()
time.sleep(2)

video=driver.find_element_by_xpath('//*[@id="iframepro"]')
video.click()

video_url=video.get_attribute('src')

driver.get(video_url)
time.sleep(2)


# depende del videoplayer
play=driver.find_element_by_xpath('/html/body/img')
play.click()

time.sleep(5)
mask=driver.find_element_by_xpath('/html/body/div[3]/a')
mask.click()
time.sleep(1)
play2=driver.find_element_by_xpath('//*[@id="loading"]/div')
play2.click()
time.sleep(2)

pantalla=driver.find_element_by_css_selector('#vstr > div.jw-wrapper.jw-reset > div.jw-controls.jw-reset > div.jw-controlbar.jw-reset > div.jw-reset.jw-button-container > div.jw-icon.jw-icon-inline.jw-button-color.jw-reset.jw-icon-fullscreen')
pantalla.click()


tiempo_cap=0
time.sleep(20)

while 1:
    tiempo_cap=driver.find_element_by_class_name('jw-icon.jw-icon-inline.jw-text.jw-reset.jw-text-countdown').get_attribute('innerText')
    
    if tiempo_cap=='00:00': break
        
video=driver.find_element_by_xpath('//*[@id="vstr"]/div[2]/div[3]/video')

AC(driver).double_click(video).perform()  # doble click, quita la pantalla completa

driver.back()
time.sleep(1)
driver.back()


pa_lante=driver.find_element_by_xpath('//*[@id="esp-option"]')
pa_lante.click()




In [18]:
driver.page_source[:100]

driver.quit()

### Ejemplo Betfair

https://www.betfair.com/sport/inplay

In [19]:
from bs4 import BeautifulSoup as bs

In [20]:
url='https://www.betfair.com/sport/inplay'

driver=webdriver.Chrome(PATH)

driver.get(url)

In [21]:
MARKET='Both teams to Score?'

In [22]:
# accept cookies
accept=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')))
accept.click()
print('Cookies accepted.')

# select lenguage
lang=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'ssc-hlsw')))
lang.click()
lang_chooser=driver.find_element_by_class_name('ssc-en_GB')
lang_chooser.click()
time.sleep(3)
print('Lenguage selected.')

print('Selecting market...')

# click in dropdown selector
dropdown=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'dropdown-1')))
dropdown.click()

# choosing bet market
chooser=WebDriverWait(dropdown, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),'+'"'+str(MARKET)+'"'+')]')))
chooser.click()

Cookies accepted.
Lenguage selected.
Selecting market...


In [23]:
results=[]

soup=bs(driver.page_source, 'html.parser')   

table=soup.find('ul', class_='section-list')

sections=table.find_all('li', class_='section')


for section in sections: # loop for each section

    league=section.find('a', class_='competition-heading').text.strip()

    games=section.select('li[class*="com-coupon-line"]')


    for game in games:   # loop for each game

        odds=game.select('span[class*="ui-runner-price"]')
        odd_1=odds[0].text.strip()
        odd_2=odds[1].text.strip()

        event_info=game.select('div[class*="event-name-info"]')[0]

        goals_team_a=event_info.find('span', class_='ui-score-home').text.strip()
        goals_team_b=event_info.find('span', class_='ui-score-away').text.strip()

        res_dict={'bookie': 'betfair',
                  'play_time': event_info.select('span[class*="event-inplay-state"]')[0].text.strip(),
                  'league': league,
                  'market': MARKET,

                  'team_a': event_info.find('span', class_='home-team-name').text.strip(),
                  'team_b': event_info.find('span', class_='away-team-name').text.strip(),
                  'goals_team_a': int(goals_team_a) if goals_team_a!='' else 0,
                  'goals_team_b': int(goals_team_b) if goals_team_b!='' else 0,

                  'odd_1': float(odd_1) if odd_1!='' else 0,
                  'odd_2': float(odd_2) if odd_2!='' else 0
                }


        results.append(res_dict)

In [24]:
results

[{'bookie': 'betfair',
  'play_time': '37′',
  'league': "Spanish Women's Matches",
  'market': 'Both teams to Score?',
  'team_a': 'Levante UD (W)',
  'team_b': 'Madrid (W)',
  'goals_team_a': 2,
  'goals_team_b': 0,
  'odd_1': 1.83,
  'odd_2': 1.83},
 {'bookie': 'betfair',
  'play_time': '36′',
  'league': 'Belarusian Premier League',
  'market': 'Both teams to Score?',
  'team_a': 'Dinamo Minsk',
  'team_b': 'Neman Grodno',
  'goals_team_a': 2,
  'goals_team_b': 0,
  'odd_1': 2.2,
  'odd_2': 1.6},
 {'bookie': 'betfair',
  'play_time': '37′',
  'league': 'Belarusian Premier League',
  'market': 'Both teams to Score?',
  'team_a': 'Energetik - BGU Minsk',
  'team_b': 'Slavia Mozyr',
  'goals_team_a': 0,
  'goals_team_b': 1,
  'odd_1': 1.52,
  'odd_2': 2.35},
 {'bookie': 'betfair',
  'play_time': '36′',
  'league': 'Belarusian Premier League',
  'market': 'Both teams to Score?',
  'team_a': 'Torpedo BelAZ',
  'team_b': 'BATE Borisov',
  'goals_team_a': 0,
  'goals_team_b': 0,
  'odd_1'